# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [42]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [43]:
# checking current working directory
print(os.getcwd())

# Get current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
    
# join the file path and roots with the subdirectories using glob
    file_path_list = glob.glob(os.path.join(root,'*'))
    #print(file_path_list)

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [44]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [45]:
# check the number of rows in csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [46]:
# This should make a connection to a Cassandra instance on local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

#### Create Keyspace

In [47]:
#Create Keyspace
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity
    WITH REPLICATION = 
    {'class' : 'SimpleStrategy', 'replication_factor' : 1}
    """)
except Exception as e:
    print(e)


#### Set Keyspace

In [48]:
#Set KEYSPACE to the keyspace specified above
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




<h3> Query 1: Return artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4</h3>

In [49]:
#Create table music_library
query = "CREATE TABLE IF NOT EXISTS music_library"
query = query + "(sessionId int, itemInSession int, artist text, song text, length float, PRIMARY KEY(sessionId, itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)       

In [50]:
#Create output CSV file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
#Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_library (sessionId, itemInSession, artist, song, length)"
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        #Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

#### Do a SELECT to verify that the data have been inserted into each table

<h5> Using the sessionID and itemInSession columns as the primary key, return the artist name, song title and song length from the music library where sessionID is 338 and itemInSession is 4. sessionID is the partitioning key and sessionID is the clustering key. Partitioning is done by sessionID and rows are ordered by itemInSession within the partition.</h5>

In [51]:
#Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, length FROM music_library WHERE sessionId = 338 AND itemInSession = 4"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.length)

Faithless Music Matters (Mark Knight Dub) 495.30731201171875


<h3> Query 2: Return name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182 </h3>

In [52]:
#Create table song_playlist_session
query = "CREATE TABLE IF NOT EXISTS song_playlist_session"
query = query + "(userId int, sessionId int, itemInSession int, artist text, song text, firstName text, lastName text, PRIMARY KEY((userId, sessionId), itemInSession))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

In [53]:
#Create output CSV file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Assign the INSERT statements into the `query` variable
        query = "INSERT INTO song_playlist_session (userId, sessionId, itemInSession, artist, song, firstName, lastName)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
        
        #Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (int(line [10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))

<h5> Using the userID and sessionID columns as the primary key and itemInSession column as the clustering key, return the artist name, song title and the user's fist and last names from the song_playlist_session table where userID is 10 and sessionID is 182. The songs are sorted by itemInSession. userID is the partitioning key and sessionID is the clustering key. Partitioning is done by userID and rows are ordered by itemInSession within the partition.</h5>

In [54]:
#Add in the SELECT statement to verify the data was entered into the table
query = "SELECT artist, song, firstName, lastName FROM song_playlist_session WHERE userId =10 AND sessionId = 182 ORDER BY itemInSession"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    print(row.artist, row.song, row.firstname, row.lastname)

Down To The Bone Keep On Keepin' On Sylvie Cruz
Three Drives Greece 2000 Sylvie Cruz
Sebastien Tellier Kilometer Sylvie Cruz
Lonnie Gordon Catch You Baby (Steve Pitron & Max Sanna Radio Edit) Sylvie Cruz


<h3> Query 3: Return every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'</h3>

In [55]:
#Create table music_app_history
query = "CREATE TABLE IF NOT EXISTS music_app_history"
query = query + "(song text, firstName text, lastName text, userID int, PRIMARY KEY(song, userID))"
try:
    session.execute(query)
except Exception as e:
    print(e)  

                    

In [56]:
#Create output CSV file. 
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        #Assign the INSERT statements into the `query` variable
        query = "INSERT INTO music_app_history (song, firstName, lastName, userID)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        #Assign which column element should be assigned for each column in the INSERT statement.
        session.execute(query, (line[9], line[1], line[4], int(line[10])))

<h5> Using the song column as the primary key and userID as the clustering key, return the user's fist and last names from the music_app_library table where the song is "All Hands Against His Own. song is the partitioning key and userID is the clustering key. Partitioning is done by song and rows are ordered by userID within the partition.</h5>

In [57]:
#Add in the SELECT statement to verify the data was entered into the table
query = "SELECT firstName, lastName FROM music_app_history WHERE song = 'All Hands Against His Own' GROUP BY userID"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
for row in rows:
    #print(row)
    print(row.firstname, row.lastname)

Jacqueline Lynch
Tegan Levine
Sara Johnson


### Drop the tables before closing out the sessions

In [58]:
#Drop the table before closing out the sessions
query = "DROP TABLE music_library"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE song_playlist_session"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)
    
query = "DROP TABLE music_app_history"
try:
    rows = session.execute(query)
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [59]:
session.shutdown()
cluster.shutdown()